In [104]:
import numpy as np
import pandas as pd
import os
import gzip
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch

In [2]:
df_stru = pd.read_csv("E:/CNSI_project/demo_and_data_3.0/Raw_data/mimic3_filtered_baidu_addICD_resp_pain.csv")
df_stru

,subject_id,hadm_id,chartdate,BLOOD GAS~BLOOD~SPECIMEN TYPE~50800~nan,Blood Gas~Blood~Alveolar-arterial Gradient~50801~_mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L_,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L_MEQ/L_,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L_",...,Hematology~Cerebrospinal Fluid (CSF)~Polys~51360~%,"Hematology~Cerebrospinal Fluid (CSF)~RBC, CSF~51362~#/uL","Hematology~Cerebrospinal Fluid (CSF)~WBC, CSF~51363~#/uL",Hematology~Cerebrospinal Fluid (CSF)~Young~51364~%,gender,age,culture,long_title,resprate,pain
0,31,128652.0,2108-08-23,NaN,NaN,0.0,NaN,24.0,NaN,NaN,...,37.0,4.0,4.0,NaN,M,72,0,Antineoplastic and immunosuppressive drugs cau...,12.076923,NaN
1,78,100536.0,2177-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,6.0,0.0,NaN,M,49,0,Accidental poisoning by benzodiazepine-based t...,17.066667,NaN
2,92,142807.0,2122-12-17,NaN,NaN,-5.0,NaN,22.0,NaN,NaN,...,0.0,94.0,0.0,NaN,F,0,0,Acidosis; Anemia of prematurity; Chronic respi...,NaN,NaN
3,93,160481.0,2128-03-23,NaN,NaN,-5.0,NaN,24.0,NaN,NaN,...,43.0,42.0,2.0,NaN,F,0,0,25-26 completed weeks of gestation; Anemia of ...,NaN,NaN
4,110,154943.0,2110-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,153.0,7.0,NaN,M,0,0,Need for prophylactic vaccination and inoculat...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111,32786,165351.0,2152-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85.0,51500.0,1500.0,NaN,M,53,0,Acute respiratory failure; Atrial fibrillation...,22.065217,NaN
2112,32786,165351.0,2152-11-21,NaN,NaN,3.0,NaN,27.0,NaN,NaN,...,69.0,29600.0,200.0,NaN,M,53,0,Acute respiratory failure; Atrial fibrillation...,23.571429,NaN
2113,32786,165351.0,2152-11-23,NaN,NaN,1.0,NaN,26.0,NaN,NaN,...,63.0,43050.0,1188.0,NaN,M,53,0,Acute respiratory failure; Atrial fibrillation...,25.758621,NaN
2114,32786,165351.0,2152-11-25,NaN,NaN,1.0,NaN,26.0,NaN,NaN,...,63.0,1130.0,96.0,NaN,M,53,0,Acute respiratory failure; Atrial fibrillation...,23.560000,NaN


In [3]:
filename = 'E:CNSI_project/demo_and_data_3.0/Raw_data/mimic-iii-clinical-database-carevue-subset-1.4/NOTEEVENTS.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    df_event = pd.read_csv(file)
df_event

C:\Users\AaronChen\AppData\Local\Temp\ipykernel_6704\3632815017.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_event = pd.read_csv(file)


,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,738407,20409,NaN,2119-01-04 00:00:00,2119-01-04 12:59:00,NaN,Radiology,ABDOMEN U.S. (COMPLETE STUDY),NaN,NaN,[**2119-1-4**] 12:59 PM\n ABDOMEN U.S. (COMPLE...
1,738408,20409,NaN,2119-01-09 00:00:00,2119-01-09 13:05:00,NaN,Radiology,MR LIVER WITH CONTRAST,NaN,NaN,[**2119-1-9**] 1:05 PM\n MR LIVER WITH CONTRAS...
2,738409,20409,NaN,2119-01-16 00:00:00,2119-01-16 21:24:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,[**2119-1-16**] 9:24 PM\n CHEST (PORTABLE AP) ...
3,738410,20409,NaN,2119-01-18 00:00:00,2119-01-18 13:24:00,NaN,Radiology,CT ABD W&W/O C,NaN,NaN,[**2119-1-18**] 1:24 PM\n CT ABD W&W/O C; CT P...
4,738411,20409,NaN,2119-01-18 00:00:00,2119-01-18 15:45:00,NaN,Radiology,PARACENTESIS DIAG. OR THERAPEUTIC,NaN,NaN,[**2119-1-18**] 3:45 PM\n PARACENTESIS DIAG. O...
...,...,...,...,...,...,...,...,...,...,...,...
880102,2083175,32806,104049.0,2162-11-02 00:00:00,2162-11-02 13:09:00,2162-11-02 13:27:00,Nursing/other,Report,14461.0,NaN,1. FEN\nTF= min140cc/k/d of BM/E24. min47cc q4...
880103,2083176,32806,104049.0,2162-11-02 00:00:00,2162-11-02 13:09:00,2162-11-02 16:36:00,Nursing/other,Report,16865.0,NaN,I have examined pt. & agree w/ [**First Name8 ...
880104,2083177,32806,104049.0,2162-11-03 00:00:00,2162-11-03 01:57:00,2162-11-03 02:13:00,Nursing/other,Report,17604.0,NaN,NPN nights\n\n\nFluids/Nutrition: Weight 2025...
880105,2083178,32806,104049.0,2162-11-03 00:00:00,2162-11-03 09:55:00,2162-11-03 09:59:00,Nursing/other,Report,21222.0,NaN,Attending Note\nDay of life 12 PMA 35 [**2-7**...


In [144]:
df_event['chartdate'] = df_event['chartdate'].str[:10]
df_event

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,738407,20409,NaN,2119-01-04,2119-01-04 12:59:00,NaN,Radiology,ABDOMEN U.S. (COMPLETE STUDY),NaN,NaN,[**2119-1-4**] 12:59 PM\n ABDOMEN U.S. (COMPLE...
1,738408,20409,NaN,2119-01-09,2119-01-09 13:05:00,NaN,Radiology,MR LIVER WITH CONTRAST,NaN,NaN,[**2119-1-9**] 1:05 PM\n MR LIVER WITH CONTRAS...
2,738409,20409,NaN,2119-01-16,2119-01-16 21:24:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,[**2119-1-16**] 9:24 PM\n CHEST (PORTABLE AP) ...
3,738410,20409,NaN,2119-01-18,2119-01-18 13:24:00,NaN,Radiology,CT ABD W&W/O C,NaN,NaN,[**2119-1-18**] 1:24 PM\n CT ABD W&W/O C; CT P...
4,738411,20409,NaN,2119-01-18,2119-01-18 15:45:00,NaN,Radiology,PARACENTESIS DIAG. OR THERAPEUTIC,NaN,NaN,[**2119-1-18**] 3:45 PM\n PARACENTESIS DIAG. O...
...,...,...,...,...,...,...,...,...,...,...,...
880102,2083175,32806,104049.0,2162-11-02,2162-11-02 13:09:00,2162-11-02 13:27:00,Nursing/other,Report,14461.0,NaN,1. FEN\nTF= min140cc/k/d of BM/E24. min47cc q4...
880103,2083176,32806,104049.0,2162-11-02,2162-11-02 13:09:00,2162-11-02 16:36:00,Nursing/other,Report,16865.0,NaN,I have examined pt. & agree w/ [**First Name8 ...
880104,2083177,32806,104049.0,2162-11-03,2162-11-03 01:57:00,2162-11-03 02:13:00,Nursing/other,Report,17604.0,NaN,NPN nights\n\n\nFluids/Nutrition: Weight 2025...
880105,2083178,32806,104049.0,2162-11-03,2162-11-03 09:55:00,2162-11-03 09:59:00,Nursing/other,Report,21222.0,NaN,Attending Note\nDay of life 12 PMA 35 [**2-7**...


In [146]:
unique_hadm = df_stru.drop_duplicates(subset=['subject_id','hadm_id','chartdate'], keep='first')
event_inStru = unique_hadm[['subject_id','hadm_id','chartdate']].merge(df_event, on=['subject_id','hadm_id','chartdate'], how='left')
event_inStru = event_inStru[['subject_id','hadm_id','chartdate','text']]
event_inStru

,subject_id,hadm_id,chartdate,text
0,31,128652.0,2108-08-23,MICU NURSING PROGRESS NOTE.\n SEE CAREVIEW...
1,31,128652.0,2108-08-23,"Respiratory Care\nPt. remains intubated, pt. p..."
2,31,128652.0,2108-08-23,ADDENDUM TO NURSING PROGRESS NOTE OF 7 AM TO 7...
3,78,100536.0,2177-02-14,[**2177-2-14**] 12:55 PM\n CHEST (PORTABLE AP)...
4,78,100536.0,2177-02-14,MICU ADMISSION NOTE\nplease see fhpa for detai...
...,...,...,...,...
9356,32786,165351.0,2152-11-28,Condition Update\nAssessment:\nPlease see care...
9357,32786,165351.0,2152-11-28,RESP CARE: Pt remains intubated with 8.0 ETT/2...
9358,32786,165351.0,2152-11-28,RESP CARE: AM RSBI-51.5. Plan is for ? trach t...
9359,32786,165351.0,2152-11-28,NPN (SEE CAREVUE FOR SPECIFICS)\n[**Name (NI) ...


In [147]:
event_inStru1 = (event_inStru.groupby(['subject_id','hadm_id','chartdate'])['text']
            .apply(lambda x: "; ".join(np.unique(x.dropna().astype(str))))
            .reset_index())
event_inStru1

,subject_id,hadm_id,chartdate,text
0,31,128652.0,2108-08-23,ADDENDUM TO NURSING PROGRESS NOTE OF 7 AM TO 7...
1,78,100536.0,2177-02-14,MICU ADMISSION NOTE\nplease see fhpa for detai...
2,92,142807.0,2122-12-17,1900-0730 NPN\nSepsis: Today is day 4 of 10 d...
3,93,160481.0,2128-03-23,NICU Fellow PN\nSecond course of indocin for m...
4,110,154943.0,2110-06-02,#1 SEPSIS\ns/o: Baby cont on ampi and gent. RE...
...,...,...,...,...
2110,32786,165351.0,2152-11-19,NURSING\n VSS OVERNIGHT. CONTINUES ON NICA...
2111,32786,165351.0,2152-11-21,Addendum to NPN:\nRight subclavian triple lume...
2112,32786,165351.0,2152-11-23,NPN (SEE CAREVUE FOR SPECIFICS)\nNEURO- PROPOF...
2113,32786,165351.0,2152-11-25,FOCUS: CONDITION UPDATE\nD: SEE CAREVUE FOR S...


In [148]:
event_inStru2 = event_inStru1.copy()
event_inStru2['length'] = event_inStru2['text'].apply(lambda x: len(x))
event_inStru2

,subject_id,hadm_id,chartdate,text,length
0,31,128652.0,2108-08-23,ADDENDUM TO NURSING PROGRESS NOTE OF 7 AM TO 7...,3021
1,78,100536.0,2177-02-14,MICU ADMISSION NOTE\nplease see fhpa for detai...,3740
2,92,142807.0,2122-12-17,1900-0730 NPN\nSepsis: Today is day 4 of 10 d...,5978
3,93,160481.0,2128-03-23,NICU Fellow PN\nSecond course of indocin for m...,11617
4,110,154943.0,2110-06-02,#1 SEPSIS\ns/o: Baby cont on ampi and gent. RE...,6760
...,...,...,...,...,...
2110,32786,165351.0,2152-11-19,NURSING\n VSS OVERNIGHT. CONTINUES ON NICA...,4339
2111,32786,165351.0,2152-11-21,Addendum to NPN:\nRight subclavian triple lume...,13712
2112,32786,165351.0,2152-11-23,NPN (SEE CAREVUE FOR SPECIFICS)\nNEURO- PROPOF...,2126
2113,32786,165351.0,2152-11-25,FOCUS: CONDITION UPDATE\nD: SEE CAREVUE FOR S...,2130


In [15]:
unique_hadm = df_stru.drop_duplicates(subset=['subject_id','hadm_id'], keep='first')
event_inStru = unique_hadm[['subject_id','hadm_id']].merge(df_event, on=['subject_id','hadm_id'], how='left')
event_inStru = event_inStru[['subject_id','hadm_id','text']]
event_inStru

,subject_id,hadm_id,text
0,31,128652.0,MICU NURSING PROGRESS NOTE.\n SEE CAREVIEW...
1,31,128652.0,"Respiratory Care\nPt. remains intubated, pt. p..."
2,31,128652.0,ADDENDUM TO NURSING PROGRESS NOTE OF 7 AM TO 7...
3,31,128652.0,NPN MICU A (NSICU Pt.) 7p-7a\n\n72 yo male adm...
4,31,128652.0,RESP CARE NOTE\nREMAINS INTUBATED ON VENT IN C...
...,...,...,...
159591,32786,165351.0,"focus update note\nafebrile, heart rate 70s, s..."
159592,32786,165351.0,RESPIRATORY CARE: PT W/ AN 8.0 PORTEX TRACH IN...
159593,32786,165351.0,npn\nNeuro- see neuro flow sheet for eval. pt ...
159594,32786,165351.0,"Resp Care Note, Pt remains t-collar 50%. Sucti..."


In [17]:
event_inStru1 = (event_inStru.groupby(['subject_id','hadm_id'])['text']
            .apply(lambda x: "; ".join(np.unique(x.dropna().astype(str))))
            .reset_index())
event_inStru1

,subject_id,hadm_id,text
0,31,128652.0,"(Continued)\nb/cardiov), no change in current ..."
1,78,100536.0,MICU ADMISSION NOTE\nplease see fhpa for detai...
2,92,142807.0,#1Sepsis\nBaby cont on amp and cefataxime. Day...
3,93,160481.0,0700-[**2052**] ADDENDUM TO NPN\n\n\nRESP: Ven...
4,110,154943.0,#1 SEPSIS\ns/o: Baby cont on ampi and gent. RE...
...,...,...,...
1371,32626,149195.0,"NPN: 7p-7a\n\nNeuro: Alert and oriented x 3, P..."
1372,32656,153884.0,#1 PT CONT ON RA. LS ARE CLEAR AND EQUAL. RR 4...
1373,32745,145060.0,"(Continued)\n STATUS. CONT TF, MONITOR U/O, SK..."
1374,32749,138488.0,0700-1900\nSEE CAREVUE FOR ASSESSMENT AND VITA...


In [150]:
# Load pre-trained biomedical NER models
model_name = "G:/BertModels/biomedical-ner-all"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a Named Entity Recognition (NER) pipeline that specifies GPU devices
nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=device)

# Define the length of the text intercept
text_max = 5000

# Define the maximum length of each block
max_length = 500  # Don't exceed 512.

In [151]:
def CC_extraction(text, text_type):

    def chunk_text(text, tokenizer, max_length):
        tokens = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=text_max)
        input_ids = tokens['input_ids'][0].tolist()
        chunks = []
        for i in range(0, len(input_ids), max_length - 2):  # Reserve space for CLS and SEP tags
            chunk = input_ids[i:i + max_length - 2]
            chunk = [tokenizer.cls_token_id] + chunk + [tokenizer.sep_token_id]
            chunks.append(chunk)
        return chunks

    chunks = chunk_text(text, tokenizer, max_length)
    # Use the NER pipeline to extract entities for each block
    entities = []
    for chunk in chunks:
        chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
        entities.extend(nlp(chunk_text))

    # Define the class of entities to be extracted. eg, Sign_symptom corresponds to the name in the model and status is a custom name.
    entity_labels = {
        "Sign_symptom": "status",
        "Detailed_description": "details",
        "Severity": "severity",
        "Medication": "medications"
    }

    results = {label: [] for label in entity_labels.values()}

    # Merge subword tags and extract entities
    current_entity = ""
    current_label = ""
    for entity in entities:
        entity_text = entity['word'].replace('##', '')
        entity_label = entity['entity_group']

        if entity_label in entity_labels:
            if entity_label == current_label:
                if entity['word'].startswith('##'):
                    current_entity += entity_text
                else:
                    current_entity += ' ' + entity_text
            else:
                if current_label and current_entity:
                    results[entity_labels[current_label]].append(current_entity.strip())
                current_entity = entity_text
                current_label = entity_label

    # Processing of the last entity
    if current_label and current_entity and current_label in entity_labels:
        results[entity_labels[current_label]].append(current_entity.strip())

    # Extracted full text
    message = ', '.join(results[text_type])

    return message

In [152]:
event_inStru2 = event_inStru1.copy()
event_inStru2['Chief Complaint'] = event_inStru2['text'].apply(lambda x: CC_extraction(x, "status"))
event_inStru2['acuity'] = event_inStru2['text'].apply(lambda x: CC_extraction(x, "severity"))
event_inStru2

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
D:\anaconda\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
D:\anaconda\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,subject_id,hadm_id,chartdate,text,Chief Complaint,acuity
0,31,128652.0,2108-08-23,ADDENDUM TO NURSING PROGRESS NOTE OF 7 AM TO 7...,"movement pupils sz activity, s sz, s, sinifica...",
1,78,100536.0,2177-02-14,MICU ADMISSION NOTE\nplease see fhpa for detai...,void hyper hypertens let lethargic sleepy neur...,"mild moderate, mild"
2,92,142807.0,2122-12-17,1900-0730 NPN\nSepsis: Today is day 4 of 10 d...,"whiteions, edema, br abrasion, hyperirubmia, h...","mild, mild"
3,93,160481.0,2128-03-23,NICU Fellow PN\nSecond course of indocin for m...,"mumur sl jaundice, opening, hyperirub murmur, ...",moderate
4,110,154943.0,2110-06-02,#1 SEPSIS\ns/o: Baby cont on ampi and gent. RE...,"sis rash, discomfort sep, rash, void stool sep...",
...,...,...,...,...,...,...
2110,32786,165351.0,2152-11-19,NURSING\n VSS OVERNIGHT. CONTINUES ON NICA...,"opens moving, stool distended, moving, opens o...",
2111,32786,165351.0,2152-11-21,Addendum to NPN:\nRight subclavian triple lume...,"hypertension, sit lethargic gag cough, ed di m...","moderate, moderate"
2112,32786,165351.0,2152-11-23,NPN (SEE CAREVUE FOR SPECIFICS)\nNEURO- PROPOF...,"alert, hyper, tachypen, wax",
2113,32786,165351.0,2152-11-25,FOCUS: CONDITION UPDATE\nD: SEE CAREVUE FOR S...,"spontaneous movements open eyes, drainia febril",small amounts


In [154]:
df_stru1 = df_stru.merge(event_inStru2[['subject_id','hadm_id','chartdate','Chief Complaint']], on=['subject_id','hadm_id','chartdate'], how='left')
df_stru1

,subject_id,hadm_id,chartdate,BLOOD GAS~BLOOD~SPECIMEN TYPE~50800~nan,Blood Gas~Blood~Alveolar-arterial Gradient~50801~_mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L_,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L_MEQ/L_,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L_",...,"Hematology~Cerebrospinal Fluid (CSF)~RBC, CSF~51362~#/uL","Hematology~Cerebrospinal Fluid (CSF)~WBC, CSF~51363~#/uL",Hematology~Cerebrospinal Fluid (CSF)~Young~51364~%,gender,age,culture,long_title,resprate,pain,Chief Complaint
0,31,128652.0,2108-08-23,NaN,NaN,0.0,NaN,24.0,NaN,NaN,...,4.0,4.0,NaN,M,72,0,Antineoplastic and immunosuppressive drugs cau...,12.076923,NaN,"movement pupils sz activity, s sz, s, sinifica..."
1,78,100536.0,2177-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,0.0,NaN,M,49,0,Accidental poisoning by benzodiazepine-based t...,17.066667,NaN,void hyper hypertens let lethargic sleepy neur...
2,92,142807.0,2122-12-17,NaN,NaN,-5.0,NaN,22.0,NaN,NaN,...,94.0,0.0,NaN,F,0,0,Acidosis; Anemia of prematurity; Chronic respi...,NaN,NaN,"whiteions, edema, br abrasion, hyperirubmia, h..."
3,93,160481.0,2128-03-23,NaN,NaN,-5.0,NaN,24.0,NaN,NaN,...,42.0,2.0,NaN,F,0,0,25-26 completed weeks of gestation; Anemia of ...,NaN,NaN,"mumur sl jaundice, opening, hyperirub murmur, ..."
4,110,154943.0,2110-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,153.0,7.0,NaN,M,0,0,Need for prophylactic vaccination and inoculat...,NaN,NaN,"sis rash, discomfort sep, rash, void stool sep..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111,32786,165351.0,2152-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51500.0,1500.0,NaN,M,53,0,Acute respiratory failure; Atrial fibrillation...,22.065217,NaN,"opens moving, stool distended, moving, opens o..."
2112,32786,165351.0,2152-11-21,NaN,NaN,3.0,NaN,27.0,NaN,NaN,...,29600.0,200.0,NaN,M,53,0,Acute respiratory failure; Atrial fibrillation...,23.571429,NaN,"hypertension, sit lethargic gag cough, ed di m..."
2113,32786,165351.0,2152-11-23,NaN,NaN,1.0,NaN,26.0,NaN,NaN,...,43050.0,1188.0,NaN,M,53,0,Acute respiratory failure; Atrial fibrillation...,25.758621,NaN,"alert, hyper, tachypen, wax"
2114,32786,165351.0,2152-11-25,NaN,NaN,1.0,NaN,26.0,NaN,NaN,...,1130.0,96.0,NaN,M,53,0,Acute respiratory failure; Atrial fibrillation...,23.560000,NaN,"spontaneous movements open eyes, drainia febril"


In [137]:
event_inStru2['Chief Complaint'][0]

'neumoboot edematous, seizure activity, spontaneous breaths, edema, spikes, seizure, spum sinusitis, sp, hypothmic, withdraw other movement pupilsz, sz, lung, hotension bradycard, m hard stool equal sluggish sp, spikes residual, edema seizure activity abd distentionffusions s, involvementsive sz desaturation, intact red areas, ns'

In [157]:
df_stru1.hadm_id.value_counts()

176676.0    17
166902.0    17
111170.0    16
125235.0    16
148986.0    14
            ..
165295.0     1
188355.0     1
114142.0     1
161579.0     1
107873.0     1
Name: hadm_id, Length: 1376, dtype: int64

In [155]:
filename = 'E:CNSI_project/demo_and_data_3.0/Raw_data/mimic-iii-clinical-database-carevue-subset-1.4/ADMISSIONS.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    df_admi = pd.read_csv(file)
df_admi

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,NaN,NaN,NEWBORN,0,1
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,1
2,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN,NaN,NaN,NEWBORN,0,1
3,6,7,118037,2121-05-23 15:05:00,2121-05-27 11:57:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1
4,7,8,159514,2117-11-20 10:22:00,2117-11-24 14:20:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26831,39771,32802,128876,2126-01-28 06:51:00,2126-01-30 15:05:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,WHITE,NaN,NaN,NEWBORN,0,1
26832,39772,32803,105824,2118-02-06 17:14:00,2118-06-25 15:00:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,NaN,UNOBTAINABLE,NaN,WHITE,NaN,NaN,NEWBORN,0,1
26833,39777,32806,104049,2162-10-22 03:00:00,2162-11-03 13:50:00,NaN,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Private,NaN,JEWISH,NaN,WHITE,NaN,NaN,NEWBORN,0,1
26834,39778,32807,140728,2173-10-27 13:00:00,2173-11-02 13:50:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,CAMB,UNOBTAINABLE,MARRIED,UNABLE TO OBTAIN,NaN,NaN,CORONARY ARTERY DISEASE,0,1


In [163]:
df_stru2 = df_stru1.copy()
df_stru2 = df_stru2.merge(df_admi[['subject_id','hadm_id','hospital_expire_flag']], on=['subject_id','hadm_id'], how='left')
df_stru2

,subject_id,hadm_id,chartdate,BLOOD GAS~BLOOD~SPECIMEN TYPE~50800~nan,Blood Gas~Blood~Alveolar-arterial Gradient~50801~_mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L_,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L_MEQ/L_,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L_",...,"Hematology~Cerebrospinal Fluid (CSF)~WBC, CSF~51363~#/uL",Hematology~Cerebrospinal Fluid (CSF)~Young~51364~%,gender,age,culture,long_title,resprate,pain,Chief Complaint,hospital_expire_flag
0,31,128652.0,2108-08-23,NaN,NaN,0.0,NaN,24.0,NaN,NaN,...,4.0,NaN,M,72,0,Antineoplastic and immunosuppressive drugs cau...,12.076923,NaN,"movement pupils sz activity, s sz, s, sinifica...",1.0
1,78,100536.0,2177-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,M,49,0,Accidental poisoning by benzodiazepine-based t...,17.066667,NaN,void hyper hypertens let lethargic sleepy neur...,0.0
2,92,142807.0,2122-12-17,NaN,NaN,-5.0,NaN,22.0,NaN,NaN,...,0.0,NaN,F,0,0,Acidosis; Anemia of prematurity; Chronic respi...,NaN,NaN,"whiteions, edema, br abrasion, hyperirubmia, h...",0.0
3,93,160481.0,2128-03-23,NaN,NaN,-5.0,NaN,24.0,NaN,NaN,...,2.0,NaN,F,0,0,25-26 completed weeks of gestation; Anemia of ...,NaN,NaN,"mumur sl jaundice, opening, hyperirub murmur, ...",0.0
4,110,154943.0,2110-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,NaN,M,0,0,Need for prophylactic vaccination and inoculat...,NaN,NaN,"sis rash, discomfort sep, rash, void stool sep...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111,32786,165351.0,2152-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1500.0,NaN,M,53,0,Acute respiratory failure; Atrial fibrillation...,22.065217,NaN,"opens moving, stool distended, moving, opens o...",0.0
2112,32786,165351.0,2152-11-21,NaN,NaN,3.0,NaN,27.0,NaN,NaN,...,200.0,NaN,M,53,0,Acute respiratory failure; Atrial fibrillation...,23.571429,NaN,"hypertension, sit lethargic gag cough, ed di m...",0.0
2113,32786,165351.0,2152-11-23,NaN,NaN,1.0,NaN,26.0,NaN,NaN,...,1188.0,NaN,M,53,0,Acute respiratory failure; Atrial fibrillation...,25.758621,NaN,"alert, hyper, tachypen, wax",0.0
2114,32786,165351.0,2152-11-25,NaN,NaN,1.0,NaN,26.0,NaN,NaN,...,96.0,NaN,M,53,0,Acute respiratory failure; Atrial fibrillation...,23.560000,NaN,"spontaneous movements open eyes, drainia febril",0.0


In [168]:
groups2 = df_stru2.groupby(['hadm_id'])['hospital_expire_flag'].apply(lambda x: x.dropna().nunique()>1)
filtered_data2 = groups2[groups2].reset_index()[['hadm_id']]
print(f"相同的hadm_id，而出现不同结果的情况有{filtered_data2.shape[0]}个。")

相同的hadm_id，而出现不同结果的情况有0个。


In [169]:
# 只取最后的一次样本
df_stru3 = df_stru2.sort_values(by=['subject_id','hadm_id','chartdate'], ascending=True)
df_stru3 = df_stru3.drop_duplicates(subset=['subject_id','hadm_id'], keep='last')
df_stru3

,subject_id,hadm_id,chartdate,BLOOD GAS~BLOOD~SPECIMEN TYPE~50800~nan,Blood Gas~Blood~Alveolar-arterial Gradient~50801~_mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L_,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L_MEQ/L_,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L_",...,"Hematology~Cerebrospinal Fluid (CSF)~WBC, CSF~51363~#/uL",Hematology~Cerebrospinal Fluid (CSF)~Young~51364~%,gender,age,culture,long_title,resprate,pain,Chief Complaint,hospital_expire_flag
0,31,128652.0,2108-08-23,NaN,NaN,0.0,NaN,24.0,NaN,NaN,...,4.0,NaN,M,72,0,Antineoplastic and immunosuppressive drugs cau...,12.076923,NaN,"movement pupils sz activity, s sz, s, sinifica...",1.0
1,78,100536.0,2177-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,M,49,0,Accidental poisoning by benzodiazepine-based t...,17.066667,NaN,void hyper hypertens let lethargic sleepy neur...,0.0
2,92,142807.0,2122-12-17,NaN,NaN,-5.0,NaN,22.0,NaN,NaN,...,0.0,NaN,F,0,0,Acidosis; Anemia of prematurity; Chronic respi...,NaN,NaN,"whiteions, edema, br abrasion, hyperirubmia, h...",0.0
3,93,160481.0,2128-03-23,NaN,NaN,-5.0,NaN,24.0,NaN,NaN,...,2.0,NaN,F,0,0,25-26 completed weeks of gestation; Anemia of ...,NaN,NaN,"mumur sl jaundice, opening, hyperirub murmur, ...",0.0
4,110,154943.0,2110-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,NaN,M,0,0,Need for prophylactic vaccination and inoculat...,NaN,NaN,"sis rash, discomfort sep, rash, void stool sep...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105,32626,149195.0,2107-06-11,NaN,NaN,4.0,NaN,28.0,NaN,NaN,...,2.0,NaN,M,33,0,"Depressive disorder, not elsewhere classified;...",17.357143,3.4,"waking disorienteras, pain turn, propiorecepti...",0.0
2106,32656,153884.0,2186-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,NaN,M,0,0,29-30 completed weeks of gestation; Neonatal j...,NaN,NaN,"tachypnea, scr pectus a b, void overt s + sep ...",0.0
2109,32745,145060.0,2151-07-22,NaN,NaN,-2.0,NaN,19.0,NaN,NaN,...,85.0,NaN,F,64,0,Acute respiratory failure; Alkalosis; Essentia...,25.266667,NaN,"nal nonpurposeful movements, opening eyes rous...",0.0
2110,32749,138488.0,2171-10-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,M,22,0,"Acidosis; Acute kidney failure, unspecified; A...",33.040000,NaN,"pupil, asleep awake awake, cough secretions co...",1.0


In [190]:
for each in df_stru3.columns.tolist():
    if 'rbc' in each.lower():
        print(each)

Hematology~Cerebrospinal Fluid (CSF)~NRBC~51357~%
Hematology~Cerebrospinal Fluid (CSF)~RBC, CSF~51362~#/uL


In [198]:
useful_cols = ['subject_id',
 'hadm_id',
 'chartdate',
 'Chemistry~Blood~Albumin~50862~g/dL_',
 'Chemistry~Blood~Alkaline Phosphatase~50863~IU/L_',
 'Chemistry~Blood~Calcium, Total~50893~mg/dL_',
 'Chemistry~Blood~Chloride~50902~mEq/L_',
 'Chemistry~Blood~Lactate Dehydrogenase (LD)~50954~IU/L',
 'Chemistry~Blood~Phosphate~50970~mg/dL_',
 'Chemistry~Blood~Urea Nitrogen~51006~mg/dL_',
 'Hematology~Blood~Eosinophils~51200~%_',
 'Hematology~Blood~INR(PT)~51237~nan',
 'Hematology~Blood~MCV~51250~fL_',
 'Hematology~Blood~Platelet Count~51265~K/uL_',
 'Hematology~Blood~PT~51274~SECONDS_sec_',
 'Hematology~Blood~RDW~51277~%_',
 'Hematology~Blood~Red Blood Cells~51279~m/uL_',
 'Hematology~Blood~White Blood Cells~51301~K/uL_',
 'Chemistry~Cerebrospinal Fluid (CSF)~Lactate Dehydrogenase, CSF~51015~IU/L',
 'Chemistry~Cerebrospinal Fluid (CSF)~Total Protein, CSF~51018~mg/dL',
 'Hematology~Cerebrospinal Fluid (CSF)~Eosinophils~51347~%',
 'Hematology~Cerebrospinal Fluid (CSF)~Polys~51360~%',
 'Hematology~Cerebrospinal Fluid (CSF)~RBC, CSF~51362~#/uL',
 'age',
 'resprate',
 'acuity',
 'Chief Complaint',
 'long_title',
 'hospital_expire_flag']

In [199]:
df_aim2 = df_stru3.reindex(useful_cols, axis=1)
df_aim2

,subject_id,hadm_id,chartdate,Chemistry~Blood~Albumin~50862~g/dL_,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L_,"Chemistry~Blood~Calcium, Total~50893~mg/dL_",Chemistry~Blood~Chloride~50902~mEq/L_,Chemistry~Blood~Lactate Dehydrogenase (LD)~50954~IU/L,Chemistry~Blood~Phosphate~50970~mg/dL_,Chemistry~Blood~Urea Nitrogen~51006~mg/dL_,...,"Chemistry~Cerebrospinal Fluid (CSF)~Total Protein, CSF~51018~mg/dL",Hematology~Cerebrospinal Fluid (CSF)~Eosinophils~51347~%,Hematology~Cerebrospinal Fluid (CSF)~Polys~51360~%,"Hematology~Cerebrospinal Fluid (CSF)~RBC, CSF~51362~#/uL",age,resprate,acuity,Chief Complaint,long_title,hospital_expire_flag
0,31,128652.0,2108-08-23,NaN,NaN,8.1,103.0,204.0,2.2,11.0,...,63.0,NaN,37.0,4.0,72,12.076923,NaN,"movement pupils sz activity, s sz, s, sinifica...",Antineoplastic and immunosuppressive drugs cau...,1.0
1,78,100536.0,2177-02-14,3.1,129.0,8.0,106.0,NaN,3.3,9.0,...,37.0,NaN,0.0,6.0,49,17.066667,NaN,void hyper hypertens let lethargic sleepy neur...,Accidental poisoning by benzodiazepine-based t...,0.0
2,92,142807.0,2122-12-17,NaN,NaN,NaN,109.0,NaN,NaN,NaN,...,142.0,NaN,0.0,94.0,0,NaN,NaN,"whiteions, edema, br abrasion, hyperirubmia, h...",Acidosis; Anemia of prematurity; Chronic respi...,0.0
3,93,160481.0,2128-03-23,NaN,NaN,NaN,104.0,NaN,NaN,NaN,...,101.0,3.0,43.0,42.0,0,NaN,NaN,"mumur sl jaundice, opening, hyperirub murmur, ...",25-26 completed weeks of gestation; Anemia of ...,0.0
4,110,154943.0,2110-06-02,NaN,126.0,NaN,106.0,NaN,NaN,NaN,...,109.0,NaN,1.0,153.0,0,NaN,NaN,"sis rash, discomfort sep, rash, void stool sep...",Need for prophylactic vaccination and inoculat...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105,32626,149195.0,2107-06-11,NaN,NaN,9.0,96.0,NaN,3.3,16.0,...,327.0,NaN,76.0,0.0,33,17.357143,NaN,"waking disorienteras, pain turn, propiorecepti...","Depressive disorder, not elsewhere classified;...",0.0
2106,32656,153884.0,2186-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,167.0,NaN,7.0,43350.0,0,NaN,NaN,"tachypnea, scr pectus a b, void overt s + sep ...",29-30 completed weeks of gestation; Neonatal j...,0.0
2109,32745,145060.0,2151-07-22,NaN,NaN,9.0,111.0,NaN,2.8,13.0,...,35.0,NaN,74.0,8850.0,64,25.266667,NaN,"nal nonpurposeful movements, opening eyes rous...",Acute respiratory failure; Alkalosis; Essentia...,0.0
2110,32749,138488.0,2171-10-17,NaN,NaN,10.2,98.0,NaN,3.5,11.0,...,26.0,NaN,13.0,8.0,22,33.040000,NaN,"pupil, asleep awake awake, cough secretions co...","Acidosis; Acute kidney failure, unspecified; A...",1.0


In [205]:
df0 = pd.read_csv("E:/CNSI_project/demo_and_data_5.0/Processed_data/MIMIC/aim2_dataAll_filtered_NameShorted.csv")
name_shorted = df0.columns.tolist()[:-5] + df0.columns.tolist()[-1:]
name_shorted

['subject_id',
 'hadm_id',
 'chartdate',
 'B_Albumin',
 'B_AlkalinePhosphatase',
 'B_Calcium',
 'B_Chloride',
 'B_LactateDehydrogenase',
 'B_Phosphate',
 'B_UreaNitrogen',
 'B_Eosinophils',
 'B_INR(PT)',
 'B_MCV',
 'B_PlateletCount',
 'B_PT',
 'B_RDW',
 'B_RedBloodCells',
 'B_WhiteBloodCells',
 'C_LactateDehydrogenase',
 'C_Protein',
 'C_Eosinophils',
 'C_Polys',
 'C_RBC',
 'age',
 'resprate',
 'acuity',
 'Chief Complaint',
 'icd_title',
 'hospital_expire_flag']

In [213]:
df_shorted = df_aim2.rename(columns=dict(zip(useful_cols, name_shorted)), inplace=False)
df_shorted = df_shorted.dropna(subset=['hospital_expire_flag'], inplace=False)
df_shorted['hospital_expire_flag'] = df_shorted['hospital_expire_flag'].astype(np.int16)
df_shorted

,subject_id,hadm_id,chartdate,B_Albumin,B_AlkalinePhosphatase,B_Calcium,B_Chloride,B_LactateDehydrogenase,B_Phosphate,B_UreaNitrogen,...,C_Protein,C_Eosinophils,C_Polys,C_RBC,age,resprate,acuity,Chief Complaint,icd_title,hospital_expire_flag
0,31,128652.0,2108-08-23,NaN,NaN,8.1,103.0,204.0,2.2,11.0,...,63.0,NaN,37.0,4.0,72,12.076923,NaN,"movement pupils sz activity, s sz, s, sinifica...",Antineoplastic and immunosuppressive drugs cau...,1
1,78,100536.0,2177-02-14,3.1,129.0,8.0,106.0,NaN,3.3,9.0,...,37.0,NaN,0.0,6.0,49,17.066667,NaN,void hyper hypertens let lethargic sleepy neur...,Accidental poisoning by benzodiazepine-based t...,0
2,92,142807.0,2122-12-17,NaN,NaN,NaN,109.0,NaN,NaN,NaN,...,142.0,NaN,0.0,94.0,0,NaN,NaN,"whiteions, edema, br abrasion, hyperirubmia, h...",Acidosis; Anemia of prematurity; Chronic respi...,0
3,93,160481.0,2128-03-23,NaN,NaN,NaN,104.0,NaN,NaN,NaN,...,101.0,3.0,43.0,42.0,0,NaN,NaN,"mumur sl jaundice, opening, hyperirub murmur, ...",25-26 completed weeks of gestation; Anemia of ...,0
4,110,154943.0,2110-06-02,NaN,126.0,NaN,106.0,NaN,NaN,NaN,...,109.0,NaN,1.0,153.0,0,NaN,NaN,"sis rash, discomfort sep, rash, void stool sep...",Need for prophylactic vaccination and inoculat...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105,32626,149195.0,2107-06-11,NaN,NaN,9.0,96.0,NaN,3.3,16.0,...,327.0,NaN,76.0,0.0,33,17.357143,NaN,"waking disorienteras, pain turn, propiorecepti...","Depressive disorder, not elsewhere classified;...",0
2106,32656,153884.0,2186-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,167.0,NaN,7.0,43350.0,0,NaN,NaN,"tachypnea, scr pectus a b, void overt s + sep ...",29-30 completed weeks of gestation; Neonatal j...,0
2109,32745,145060.0,2151-07-22,NaN,NaN,9.0,111.0,NaN,2.8,13.0,...,35.0,NaN,74.0,8850.0,64,25.266667,NaN,"nal nonpurposeful movements, opening eyes rous...",Acute respiratory failure; Alkalosis; Essentia...,0
2110,32749,138488.0,2171-10-17,NaN,NaN,10.2,98.0,NaN,3.5,11.0,...,26.0,NaN,13.0,8.0,22,33.040000,NaN,"pupil, asleep awake awake, cough secretions co...","Acidosis; Acute kidney failure, unspecified; A...",1


In [215]:
df_shorted.to_excel('E:/CNSI_project/demo_and_data_5.0/Processed_data/Extr_val/aim2_dataset_ExVal.xlsx', index=False)

In [54]:
from graphviz import Digraph

def create_flowchart():
    # Create a directed graph
    dot = Digraph(format='svg')  # Change the format to 'svg' for better clarity
    dot.attr(rankdir='TB', size='8,5', ranksep='0.2')  # Add ranksep to adjust vertical spacing
    dot.attr('node', shape='rectangle', style='filled', fillcolor='white', fontname='Times', fontsize='12', height='0.3')

    # Input Node
    dot.node('A', 'Raw Clinical Text from NOTEEVENTS.csv', fillcolor='white')

    # Step 1: Text Segmentation
    dot.node('B', 'Text Segmentation (Tokenization)', fillcolor='white')
    dot.edge('A', 'B', label='     Tokenize text into manageable chunks', fontsize='12', arrowsize='0.7')

    # Step 2: Named Entity Recognition (NER)
    dot.node('C', 'Named Entity Recognition (NER) Using biomedical-ner-all', fillcolor='white')
    dot.edge('B', 'C', label='     Analyze chunks to extract entities', fontsize='12', arrowsize='0.7')

    # Step 3: Entity Mapping and Filtering
    dot.node('D', 'Entity Filtering and Mapping', fillcolor='white')
    dot.edge('C', 'D', label='     Filter entities to defined categories', fontsize='12', arrowsize='0.7')

    # Step 4: Concatenate and Compile Entities
    dot.node('E', 'Entity Compilation for Chief Complaint', fillcolor='white')
    dot.edge('D', 'E', label='     Compile extracted status entities', fontsize='12', arrowsize='0.7')

    # Output Node
    dot.node('F', 'Chief Complaint Extraction Output', shape='ellipse', fillcolor='white')
    dot.edge('E', 'F', label='     Final Chief Complaint message', fontsize='12', arrowsize='0.7')

    # Render and save the flowchart as SVG
    dot.render('chief_complaint_extraction_flowchart', view=True)

# Call the function to create the flowchart
create_flowchart()



(process:13424): Pango-WARNING **: 21:03:25.293: couldn't load font "Times Not-Rotated 12", falling back to "Sans Not-Rotated 12", expect ugly output.
